In [2]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense
import keras_tuner as kt

In [3]:
df = pd.read_csv("Datasets/diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
x = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [7]:
x = scaler.fit_transform(x)

In [8]:
x

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)

In [10]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim = 8))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

/home/petpooja--1118/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-02-07 12:24:34.834319: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [11]:
model.fit(x_train, y_train, validation_data = (x_test, y_test) ,batch_size = 32, epochs = 100)

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.6556 - loss: 0.7298 - val_accuracy: 0.6429 - val_loss: 0.6852
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6519 - loss: 0.6773 - val_accuracy: 0.6558 - val_loss: 0.6331
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6538 - loss: 0.6260 - val_accuracy: 0.6688 - val_loss: 0.5957
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6912 - loss: 0.5701 - val_accuracy: 0.6948 - val_loss: 0.5687
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6917 - loss: 0.5754 - val_accuracy: 0.7468 - val_loss: 0.5466
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7180 - loss: 0.5364 - val_accuracy: 0.7403 - val_loss: 0.5282
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7159 - loss: 0.5283 - val_accuracy: 0.7662 - val_loss: 0.5138
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7141 - loss: 0.5152 - val_accuracy: 0.7727 - v

----------

In [12]:
import keras_tuner as kt

In [13]:
def build_model(hp):
    model = Sequential()

    model.add(Dense(32, activation='relu', input_dim = 8))
    model.add(Dense(1, activation = 'sigmoid'))

    optimizer = hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta'])
    model.compile(optimizer=optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [14]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5)

In [15]:
tuner.search(x_train, y_train, epochs = 10, validation_data = (x_test, y_test))

Trial 4 Complete [00h 00m 01s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 05s


In [16]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [17]:
model = tuner.get_best_models(num_models=1)[0]

/home/petpooja--1118/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/home/petpooja--1118/.local/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [18]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.fit(x_train, y_train, epochs=100, initial_epoch=11, validation_data=(x_test, y_test))

Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7697 - loss: 0.4712 - val_accuracy: 0.7727 - val_loss: 0.4714
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7730 - loss: 0.4725 - val_accuracy: 0.7727 - val_loss: 0.4692
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7742 - loss: 0.4526 - val_accuracy: 0.7597 - val_loss: 0.4669
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7663 - loss: 0.4841 - val_accuracy: 0.7662 - val_loss: 0.4659
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7451 - loss: 0.4928 - val_accuracy: 0.7662 - val_loss: 0.4654
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7974 - loss: 0.4357 - val_accuracy: 0.7597 - val_loss: 0.4656
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7804 - loss: 0.4445 - val_accuracy: 0.7662 - val_loss: 0.4645
Epoch 19/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7522 - loss: 0.4640 - val_accuracy: 0.

-----

# Right number of neurons

In [14]:
def build_model(hp):
    model = Sequential()
    units = hp.Int('units', min_value = 8, max_value = 128, step = 8)

    model.add(Dense(units=units, activation='relu', input_dim = 8))
    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [15]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5)

2025-02-07 12:40:44.399117: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/petpooja--1118/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [16]:
tuner.search(x_train, y_train, epochs = 10, validation_data = (x_test, y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 07s


In [17]:
tuner.get_best_hyperparameters()[0].values

{'units': 120}

In [18]:
model = tuner.get_best_models(num_models=1)[0]

/home/petpooja--1118/.local/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [19]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 120)            │         1,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           121 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,201 (4.69 KB)

 Trainable params: 1,201 (4.69 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.fit(x_train, y_train, epochs=100, initial_epoch=10, validation_data=(x_test, y_test))

Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7791 - loss: 0.4543 - val_accuracy: 0.7922 - val_loss: 0.4611
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8086 - loss: 0.4381 - val_accuracy: 0.7987 - val_loss: 0.4601
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7669 - loss: 0.4509 - val_accuracy: 0.8052 - val_loss: 0.4565
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7756 - loss: 0.4389 - val_accuracy: 0.7987 - val_loss: 0.4580
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7693 - loss: 0.4671 - val_accuracy: 0.8052 - val_loss: 0.4579
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8015 - loss: 0.4128 - val_accuracy: 0.8052 - val_loss: 0.4591
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7857 - loss: 0.4239 - val_accuracy: 0.8052 - val_loss: 0.4589
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7945 - loss: 0.4296 - val_accuracy: 0.

-------

# Select number of layers

In [15]:
def build_model(hp):

    model = Sequential()

    model.add(Dense(72, activation = 'relu', input_dim = 8))

    for i in range(hp.Int('num_layers', min_value =1, max_value =10)):
        model.add(Dense(72, activation = 'relu'))

    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

    return model

In [16]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5)

In [17]:
tuner.search(x_train, y_train, epochs = 10, validation_data = (x_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.8376623392105103

Best val_accuracy So Far: 0.8376623392105103
Total elapsed time: 00h 00m 11s


In [18]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6}

In [19]:
model = tuner.get_best_models(num_models=1)[0]

/home/petpooja--1118/.local/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 34 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 72)             │           648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 72)             │         5,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            73 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,257 (126.00 KB)

 Trainable params: 32,257 (126.00 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
model.fit(x_train, y_train, epochs=100, initial_epoch=10, validation_data=(x_test, y_test))

Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7712 - loss: 0.4398 - val_accuracy: 0.7792 - val_loss: 0.4778
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8060 - loss: 0.4082 - val_accuracy: 0.8052 - val_loss: 0.4630
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7887 - loss: 0.4590 - val_accuracy: 0.7987 - val_loss: 0.4700
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8012 - loss: 0.4192 - val_accuracy: 0.8182 - val_loss: 0.4725
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8058 - loss: 0.4141 - val_accuracy: 0.8247 - val_loss: 0.4690
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8291 - loss: 0.3797 - val_accuracy: 0.8052 - val_loss: 0.4751
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8167 - loss: 0.3894 - val_accuracy: 0.7857 - val_loss: 0.4859
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8350 - loss: 0.3597 - val_accuracy: 0.

---

In [10]:
def build_model(hp):
    model = Sequential()
    
    counter = 0
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        if counter == 0:
            model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                            activation=hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])))
        else:
            model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                            activation=hp.Choice('activation', values=['relu', 'tanh', 'sigmoid'])))
            counter += 1
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop', 'adam', 'sgd', 'nadam', 'adadelta']),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [11]:
tuner = kt.RandomSearch(build_model,
                       objective='val_accuracy',
                       max_trials=5)

2025-02-07 13:59:01.965439: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [12]:
tuner.search(x_train, y_train, epochs = 10, validation_data = (x_test, y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.8181818127632141

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 08s


In [15]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4,
 'units0': 16,
 'activation': 'tanh',
 'optimizer': 'adam',
 'units1': 96,
 'units2': 48,
 'units3': 16}

In [16]:
model = tuner.get_best_models(num_models=1)[0]

/home/petpooja--1118/.local/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 96)             │         1,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 48)             │         4,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 16)             │           784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,233 (28.25 KB)

 Trainable params: 7,233 (28.25 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.fit(x_train, y_train, epochs=100, initial_epoch=10, validation_data=(x_test, y_test))

Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7741 - loss: 0.4575 - val_accuracy: 0.8052 - val_loss: 0.4732
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7795 - loss: 0.4671 - val_accuracy: 0.7987 - val_loss: 0.4662
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7675 - loss: 0.4783 - val_accuracy: 0.7727 - val_loss: 0.4735
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7791 - loss: 0.4567 - val_accuracy: 0.7792 - val_loss: 0.4709
Epoch 15/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7722 - loss: 0.4666 - val_accuracy: 0.7922 - val_loss: 0.4668
Epoch 16/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7692 - loss: 0.4517 - val_accuracy: 0.7662 - val_loss: 0.4729
Epoch 17/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7996 - loss: 0.4253 - val_accuracy: 0.7792 - val_loss: 0.4644
Epoch 18/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7871 - loss: 0.4547 - val_accuracy: 0.

# Thank you